In [67]:
import numpy as np
from io import StringIO
from collections import defaultdict
from scipy import linalg

In [68]:
m1 = """\
                1             2             3             4             5 
      1  0.100000D+01
      2  0.236704D+00  0.100000D+01
      3  0.000000D+00  0.000000D+00  0.100000D+01
      4  0.000000D+00  0.000000D+00  0.000000D+00  0.100000D+01
      5  0.000000D+00  0.000000D+00  0.000000D+00  0.000000D+00  0.100000D+01
      6  0.487518D-01  0.447147D+00  0.000000D+00  0.299042D+00 -0.231543D+00
      7  0.487518D-01  0.447147D+00  0.000000D+00 -0.299042D+00 -0.231543D+00
                6             7 
      6  0.100000D+01
      7  0.228239D+00  0.100000D+01"""

In [84]:
def gaussian_to_numpy(text):
    all_data = [x.strip().split(" ") for x in text.split("\n")]
    ragged = [row for row in all_data if len(row) <= 12]
    triangle = defaultdict(lambda : [])
    for row in ragged:
        to_append = [float(x.replace("D", "e")) for i, x in enumerate(row) if i>0 and x!=""]
        if len(to_append) >= 1:
            triangle[int(row[0])].extend(to_append)

    N = max(triangle.keys())
    M = np.zeros((N, N))
    for key, val in triangle.items():
        M[key-1, :len(val)] = val
    M_full = M+M.T-np.diag(M.diagonal())
    return M_full

In [85]:
o1 = gaussian_to_numpy(m1)

In [86]:
o1.round(3)

array([[ 1.   ,  0.237,  0.   ,  0.   ,  0.   ,  0.049,  0.049],
       [ 0.237,  1.   ,  0.   ,  0.   ,  0.   ,  0.447,  0.447],
       [ 0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.299, -0.299],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   , -0.232, -0.232],
       [ 0.049,  0.447,  0.   ,  0.299, -0.232,  1.   ,  0.228],
       [ 0.049,  0.447,  0.   , -0.299, -0.232,  0.228,  1.   ]])

In [87]:
kinetic="""\
1             2             3             4             5 
      1  0.290032D+02
      2 -0.168011D+00  0.808128D+00
      3  0.000000D+00  0.000000D+00  0.252873D+01
      4  0.000000D+00  0.000000D+00  0.000000D+00  0.252873D+01
      5  0.000000D+00  0.000000D+00  0.000000D+00  0.000000D+00  0.252873D+01
      6 -0.512016D-02  0.109023D+00  0.000000D+00  0.199564D+00 -0.154519D+00
      7 -0.512016D-02  0.109023D+00  0.000000D+00 -0.199564D+00 -0.154519D+00
                6             7 
      6  0.760032D+00
      7  0.359579D-02  0.760032D+00
"""

In [89]:
T = gaussian_to_numpy(kinetic)

In [90]:
potential="""\
1             2             3             4             5 
      1  0.616768D+02
      2  0.743360D+01  0.100996D+02
      3  0.000000D+00  0.000000D+00  0.995087D+01
      4  0.000000D+00  0.000000D+00  0.000000D+00  0.100924D+02
      5 -0.174154D-01 -0.207813D+00  0.000000D+00  0.000000D+00  0.100357D+02
      6  0.157306D+01  0.358237D+01  0.000000D+00  0.212431D+01 -0.170905D+01
      7  0.157306D+01  0.358237D+01  0.000000D+00 -0.212431D+01 -0.170905D+01
                6             7 
      6  0.566361D+01
      7  0.142625D+01  0.566361D+01
"""

In [91]:
V = gaussian_to_numpy(potential)

In [92]:
V

array([[ 6.16768e+01,  7.43360e+00,  0.00000e+00,  0.00000e+00,
        -1.74154e-02,  1.57306e+00,  1.57306e+00],
       [ 7.43360e+00,  1.00996e+01,  0.00000e+00,  0.00000e+00,
        -2.07813e-01,  3.58237e+00,  3.58237e+00],
       [ 0.00000e+00,  0.00000e+00,  9.95087e+00,  0.00000e+00,
         0.00000e+00,  0.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00924e+01,
         0.00000e+00,  2.12431e+00, -2.12431e+00],
       [-1.74154e-02, -2.07813e-01,  0.00000e+00,  0.00000e+00,
         1.00357e+01, -1.70905e+00, -1.70905e+00],
       [ 1.57306e+00,  3.58237e+00,  0.00000e+00,  2.12431e+00,
        -1.70905e+00,  5.66361e+00,  1.42625e+00],
       [ 1.57306e+00,  3.58237e+00,  0.00000e+00, -2.12431e+00,
        -1.70905e+00,  1.42625e+00,  5.66361e+00]])

In [93]:
Hcore_str="""
 1             2             3             4             5 
      1 -0.326736D+02
      2 -0.760161D+01 -0.929148D+01
      3  0.000000D+00  0.000000D+00 -0.742214D+01
      4  0.000000D+00  0.000000D+00  0.000000D+00 -0.756362D+01
      5  0.174154D-01  0.207813D+00  0.000000D+00  0.000000D+00 -0.750696D+01
      6 -0.157818D+01 -0.347335D+01  0.000000D+00 -0.192475D+01  0.155453D+01
      7 -0.157818D+01 -0.347335D+01  0.000000D+00  0.192475D+01  0.155453D+01
                6             7 
      6 -0.490358D+01
      7 -0.142266D+01 -0.490358D+01"""

In [96]:

Hcore = gaussian_to_numpy(Hcore_str)
Hcore.shape

(7, 7)

In [97]:
basis_str = """
 1             2             3             4             5 
      1   0.362238D+00  0.448049D+00  0.829144D+00  0.223948D+00  0.000000D+00
      2   0.597041D+00  0.159718D+00 -0.466225D+00 -0.102777D+01  0.000000D+00
      3   0.000000D+00  0.000000D+00  0.000000D+00  0.000000D+00  0.100000D+01
      4   0.000000D+00  0.000000D+00  0.000000D+00  0.000000D+00  0.000000D+00
      5  -0.209193D+00  0.818474D+00 -0.437341D+00  0.495173D+00  0.000000D+00
      6   0.200635D+00 -0.107707D+00 -0.141403D+00  0.792594D+00  0.000000D+00
      7   0.200635D+00 -0.107707D+00 -0.141403D+00  0.792594D+00  0.000000D+00
                 6             7 
      1   0.000000D+00  0.000000D+00
      2   0.000000D+00  0.000000D+00
      3   0.000000D+00  0.000000D+00
      4   0.874749D+00 -0.732438D+00
      5   0.000000D+00  0.000000D+00
      6   0.177786D+00  0.900939D+00
      7  -0.177786D+00 -0.900939D+00"""

In [98]:
basis0 = gaussian_to_numpy(basis_str)